In [1]:
import pandas as pd
import numpy as np

### Loading the data into a pandas dataframe: 

In [2]:
df = pd.read_csv('Data.csv')
df.head()

,id,user_id,vehicle_model_id,package_id,travel_type_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,to_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
0,132512,22177,28,NaN,2,83.0,448.0,NaN,NaN,1/1/2013 2:00,NaN,0,0,1/1/2013 1:39,12.924150,77.672290,12.927320,77.635750,0
1,132513,21413,12,NaN,2,1010.0,540.0,NaN,NaN,1/1/2013 9:00,NaN,0,0,1/1/2013 2:25,12.966910,77.749350,12.927680,77.626640,0
2,132514,22178,12,NaN,2,1301.0,1034.0,NaN,NaN,1/1/2013 3:30,NaN,0,0,1/1/2013 3:08,12.937222,77.626915,13.047926,77.597766,0
3,132515,13034,12,NaN,2,768.0,398.0,NaN,NaN,1/1/2013 5:45,NaN,0,0,1/1/2013 4:39,12.989990,77.553320,12.971430,77.639140,0
4,132517,22180,12,NaN,2,1365.0,849.0,NaN,NaN,1/1/2013 9:00,NaN,0,0,1/1/2013 7:53,12.845653,77.677925,12.954340,77.600720,0


Here, our first intuition can be to index the dataframe based on the 'from_area_id' since our goal is to optimize cab utilization by figuring out what areas, at what times get the most/least bookings and what areas get most/least cancellations. So let's first index the dataframe on 'from_area_id' since from that's where a cab is booked or the passenger is picked up:

In [3]:
df2 = df.set_index('from_area_id')
df2 = df2.sort_index()
df2

,id,user_id,vehicle_model_id,package_id,travel_type_id,to_area_id,from_city_id,to_city_id,from_date,to_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
from_area_id,,,,,,,,,,,,,,,,,,
2.0,153015,32926,12,NaN,2,393.0,NaN,NaN,6/4/2013 7:30,NaN,1,0,6/3/2013 20:44,12.99212,77.67218,13.19956,77.70688,0
2.0,161625,37064,12,NaN,2,393.0,NaN,NaN,7/20/2013 13:00,7/20/2013 14:21,1,0,7/20/2013 9:52,12.99212,77.67218,13.19956,77.70688,0
2.0,167832,39970,12,NaN,2,393.0,15.0,NaN,8/18/2013 19:00,8/18/2013 21:16,1,0,8/18/2013 16:39,12.99212,77.67218,13.19956,77.70688,0
2.0,137160,17870,12,NaN,2,222.0,NaN,NaN,2/9/2013 7:45,NaN,0,0,2/9/2013 2:03,12.99212,77.67218,12.97216,77.59383,0
2.0,139846,25975,87,2.0,3,NaN,NaN,NaN,3/3/2013 0:00,NaN,0,0,3/2/2013 19:24,12.99212,77.67218,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,183234,47299,12,NaN,1,NaN,15.0,131.0,1/6/2014 5:00,NaN,1,0,11/5/2013 15:49,NaN,NaN,NaN,NaN,0
NaN,184636,48037,65,NaN,1,NaN,15.0,69.0,11/15/2013 4:15,11/15/2013 0:00,1,0,11/15/2013 1:14,NaN,NaN,NaN,NaN,0
NaN,185226,48368,12,NaN,1,NaN,15.0,29.0,11/19/2013 18:15,11/20/2013 23:59,1,0,11/19/2013 15:15,NaN,NaN,NaN,NaN,0


<b>After indexing the dataframe on 'area_id' we can see that we have multiple rows with same 'area_id'. But We don't want that. We want to merge all the data with the same 'area_id' to a single row pointing to all the data of a particular 'area_id'.

For that, we can use .groupby() function of a dataframe & pass in the name of the column we want to group by & additionally a function agg() to it which takes a lambda function that tells how all the data values for each area_ids is  going to be represented.</b>

In [4]:
area_based_df = df2.groupby('from_area_id').agg(lambda x: list(x) if len(x) > 1 else x.iloc[0])
area_based_df

,id,user_id,vehicle_model_id,package_id,travel_type_id,to_area_id,from_city_id,to_city_id,from_date,to_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation
from_area_id,,,,,,,,,,,,,,,,,,
2.0,"[153015, 161625, 167832, 137160, 139846, 17865...","[32926, 37064, 39970, 17870, 25975, 45036, 495...","[12, 12, 12, 12, 87, 24, 12, 12, 12, 12, 12, 1...","[nan, nan, nan, nan, 2.0, nan, nan, nan, nan, ...","[2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, ...","[393.0, 393.0, 393.0, 222.0, nan, 585.0, 452.0...","[nan, nan, 15.0, nan, nan, 15.0, 15.0, nan, 15...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[6/4/2013 7:30, 7/20/2013 13:00, 8/18/2013 19:...","[nan, 7/20/2013 14:21, 8/18/2013 21:16, nan, n...","[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6/3/2013 20:44, 7/20/2013 9:52, 8/18/2013 16:...","[12.99212, 12.99212, 12.99212, 12.99212, 12.99...","[77.67218000000001, 77.67218000000001, 77.6721...","[13.19956, 13.19956, 13.19956, 12.97216, nan, ...","[77.70688, 77.70688, 77.70688, 77.593830000000...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
6.0,"[157461, 132632, 133538, 141100, 158202, 16182...","[22027, 694, 743, 33406, 743, 37177, 41719]","[12, 28, 12, 12, 12, 12, 90]","[nan, 2.0, nan, nan, nan, nan, nan]","[2, 3, 2, 2, 2, 2, 2]","[515.0, nan, 393.0, 448.0, 585.0, 393.0, 393.0]","[nan, nan, nan, nan, nan, nan, 15.0]","[nan, nan, nan, nan, nan, nan, nan]","[6/29/2013 11:45, 1/2/2013 16:00, 1/12/2013 6:...","[6/29/2013 12:17, 1/2/2013 0:00, nan, nan, 7/3...","[1, 1, 0, 0, 0, 1, 1]","[0, 0, 0, 0, 0, 0, 0]","[6/29/2013 9:27, 1/2/2013 14:34, 1/11/2013 16:...","[12.97739, 12.97739, 12.97739, 12.97739, 12.97...","[77.61762, 77.61762, 77.61762, 77.61762, 77.61...","[12.97896, nan, 13.19956, 12.92732, 12.97677, ...","[77.67345, nan, 77.70688, 77.63575, 77.5727, 7...","[0, 0, 0, 0, 0, 0, 0]"
15.0,"[160557, 152943, 141165, 143758, 152894, 135391]","[36550, 32869, 1489, 25924, 32869, 4828]","[12, 12, 1, 12, 12, 24]","[nan, nan, nan, nan, nan, nan]","[2, 2, 2, 2, 2, 2]","[393.0, 936.0, 393.0, 1390.0, 603.0, 168.0]","[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]","[7/17/2013 12:30, 6/4/2013 8:15, 3/15/2013 10:...","[7/17/2013 14:06, nan, nan, nan, nan, nan]","[0, 1, 0, 1, 1, 0]","[0, 0, 0, 0, 0, 0]","[7/14/2013 13:08, 6/3/2013 13:27, 3/15/2013 9:...","[12.960469999999999, 12.960469999999999, 12.96...","[77.5658, 77.5658, 77.5658, 77.5658, 77.5658, ...","[13.19956, 12.97251, 13.19956, 12.969368, 12.9...","[77.70688, 77.61954, 77.70688, 77.641302, 77.6...","[0, 0, 0, 0, 0, 0]"
16.0,"[154215, 168254, 168033, 137294, 139497]","[29519, 40064, 40064, 24714, 21704]","[12, 12, 12, 12, 12]","[1.0, nan, nan, nan, nan]","[3, 2, 2, 2, 2]","[nan, 1034.0, 1034.0, 1057.0, 1229.0]","[nan, 15.0, 15.0, nan, nan]","[nan, nan, nan, nan, nan]","[6/10/2013 9:00, 8/20/2013 18:00, 8/19/2013 18...","[nan, 8/20/2013 18:58, 8/19/2013 19:28, nan, nan]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[6/9/2013 20:40, 8/20/2013 15:38, 8/19/2013 15...","[12.986239999999999, 12.986239999999999, 12.98...","[77.59205, 77.59205, 77.59205, 77.59205, 77.59...","[nan, 13.047926, 13.047926, 12.916155999999999...","[nan, 77.59776600000001, 77.59776600000001, 77...","[0, 0, 0, 0, 0]"
17.0,"[165348, 165352]","[37713, 37713]","[12, 12]","[1.0, 1.0]","[3, 3]","[nan, nan]","[nan, nan]","[nan, nan]","[8/7/2013 17:45, 8/7/2013 17:45]","[8/7/2013 22:45, 8/8/2013 0:21]","[1, 1]","[0, 0]","[8/7/2013 14:43, 8/7/2013 14:50]","[13.0777, 13.0777]","[77.58164000000001, 77.58164000000001]","[nan, nan]","[nan, nan]","[0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397.0,"[182842, 182765]","[47058, 46987]","[12, 12]","[nan, nan]","[1, 1]","[nan, nan]","[31.0, 31.0]","[132.0, 132.0]","[11/3/2013 6:15, 11/3/2013 5:45]","[nan, nan]","[0, 0]","[1, 1]","[11/2/2013 13:56, 11/1/2013 19:06]","[nan, nan]","[nan, nan]","[nan, nan]","[nan, n

<b>Now, let's create a function that returns a dictionary with the area_id's as the keys and number of bookings as it values. Here we figure out number of bookings for each area based on the length of list in the 'id' column which represents all the booking_ids corresponding to each area_id.

After creating the dictionary we can now sort it based on the number of bookings in descending order (higher number of bookings to lower):</b>

In [5]:
def bookings_per_area(df):
    df = df.dropna()
    area_list = df.index
    d = {}
    for i in area_list:
        b = df.loc[i,'id']
        num = len(b)
        d[i] = num
    return d

bookings = bookings_per_area(area_based_df)
sorted_bookings = dict(sorted(bookings.items(), key=lambda item: item[1],reverse=True))
sorted_bookings

{393.0: 3858,
 571.0: 1631,
 293.0: 1052,
 585.0: 911,
 1010.0: 768,
 142.0: 727,
 83.0: 719,
 1384.0: 628,
 1096.0: 542,
 58.0: 466,
 269.0: 413,
 1026.0: 395,
 625.0: 370,
 1017.0: 363,
 1330.0: 362,
 689.0: 341,
 768.0: 334,
 271.0: 333,
 1056.0: 330,
 1371.0: 327,
 1068.0: 324,
 363.0: 308,
 396.0: 289,
 1021.0: 279,
 1365.0: 273,
 105.0: 271,
 410.0: 266,
 458.0: 261,
 515.0: 251,
 776.0: 249,
 572.0: 248,
 450.0: 246,
 1165.0: 246,
 392.0: 242,
 1095.0: 241,
 1390.0: 235,
 1084.0: 229,
 168.0: 226,
 61.0: 221,
 1054.0: 220,
 542.0: 216,
 353.0: 215,
 1089.0: 213,
 149.0: 212,
 1038.0: 212,
 793.0: 211,
 767.0: 208,
 136.0: 206,
 171.0: 198,
 949.0: 198,
 448.0: 197,
 1086.0: 197,
 977.0: 191,
 1041.0: 191,
 590.0: 189,
 1281.0: 188,
 601.0: 185,
 89.0: 184,
 330.0: 181,
 455.0: 181,
 292.0: 173,
 502.0: 171,
 1237.0: 170,
 1192.0: 169,
 150.0: 166,
 115.0: 161,
 471.0: 161,
 433.0: 157,
 218.0: 155,
 1373.0: 155,
 25.0: 153,
 297.0: 153,
 1063.0: 150,
 112.0: 149,
 1147.0: 147,
 

<b>We now store it in a new pandas series for later use:</b>

In [6]:
bookings_per_area_series = pd.Series(sorted_bookings)
bookings_per_area_series

393.0     3858
571.0     1631
293.0     1052
585.0      911
1010.0     768
          ... 
1360.0       2
1369.0       2
1381.0       2
1385.0       2
1397.0       2
Length: 560, dtype: int64

<b>Now, we create another function that returns a dictionary with the area_id's as the keys and Car cancellation as it values. Here the Car cancellations value are going to be either 'Yes' or 'No' based on whether the list of values in the 'Car_Cancellation' column have any 1s or all 0s.

We can then store it into another new series for later use:</b>

In [7]:
def cancellation_per_area(df):
    df = df.dropna()
    areas_list = df.index
    d = {}
    for i in areas_list:
        x = df.loc[i,'Car_Cancellation']
        if 1 in x:
            x = 'Yes'
        else:
            x = 'No'
        d[i] = x
    return d
car_cancellation = cancellation_per_area(area_based_df)
cancellation_per_area_series = pd.Series(car_cancellation)
cancellation_per_area_series

2.0       Yes
6.0        No
15.0       No
16.0       No
17.0       No
         ... 
1396.0     No
1397.0     No
1398.0    Yes
1399.0    Yes
1401.0    Yes
Length: 560, dtype: object

<b>Another function we can also define is to calculate the number of times a cab has been cancelled for a particular area & store in a dictionary with area_id's as the keys and number of cancellations as its values.

We'll then store that one in a separate series too:</b>

In [8]:
def cancelled_times(df):
    df = df.dropna()
    areas_list = df.index
    d = {}
    for i in areas_list:
        x = df.loc[i,'Car_Cancellation']
        y = 0
        for j in x:
            if j == 1:
                y += 1
        d[i] = y
    return d
no_of_cancelled = cancelled_times(area_based_df)
no_of_cancelled_series = pd.Series(no_of_cancelled)
no_of_cancelled_series

2.0       4
6.0       0
15.0      0
16.0      0
17.0      0
         ..
1396.0    0
1397.0    0
1398.0    3
1399.0    9
1401.0    2
Length: 560, dtype: int64

<b>Next we define a function to reduce the list of travel_ytpe_id to store a list of all the CANCELLED  'travel_type_id's for a given area_id in a dictionary & also store it in a slightly different format where 'L' means Long distance, 'P2P' means Point-to-Point travel & 'H' for Hourly basis travel.

We'll then store that resulting dictionary in a new series:</b>

In [9]:
def cancelled_ttype_per_area(df):
    df = df.dropna()
    area_list = df.index
    d = {}
    for i in area_list:
        c = df.loc[i,'travel_type_id']
        e = df.loc[i,'Car_Cancellation']
        l = []
        for index,val in enumerate(c):
            if val == 1:
                c[index] = 'L'
            if val == 2:
                c[index] = 'P2P'
            if val == 3:
                c[index] = 'H'
                
        for j,v in enumerate(e):
            if v == 1:
                l.append(c[j])
        d[i] = l
    return d

travel_ids = cancelled_ttype_per_area(area_based_df)
travel_ids_per_area_series = pd.Series(travel_ids)
travel_ids_per_area_series.tail(20)

1378.0               [P2P, P2P, H, P2P, P2P, P2P, P2P, P2P]
1379.0    [P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ...
1381.0                                                [P2P]
1382.0                                                   []
1383.0          [P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, H]
1384.0    [P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ...
1385.0                                                [P2P]
1386.0                                                [P2P]
1388.0                    [P2P, P2P, P2P, P2P, P2P, H, P2P]
1389.0                                                   []
1390.0    [P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ...
1391.0                             [P2P, P2P, H, H, P2P, H]
1393.0                                                  [H]
1394.0                                                [P2P]
1395.0                         [P2P, P2P, H, P2P, P2P, P2P]
1396.0                                                   []
1397.0                                  

<b>Now we will define functions to store starting & ending times of cab trips from the 'from_date' & 'to_date' columns into a dictionary as values corresponding to area_id's and then store them into different series:</b>

In [10]:
def bookings_per_from_time(df):
    df = df.dropna()
    area_list = df.index
    d = {}
    for i in area_list:
        c = df.loc[i,'from_date']
        d[i] = c
    return d

bookings_per_time1 = bookings_per_from_time(area_based_df)
bookings_per_time1
bookings_per_time1_series = pd.Series(bookings_per_time1)
bookings_per_time1_series

2.0       [6/4/2013 7:30, 7/20/2013 13:00, 8/18/2013 19:...
6.0       [6/29/2013 11:45, 1/2/2013 16:00, 1/12/2013 6:...
15.0      [7/17/2013 12:30, 6/4/2013 8:15, 3/15/2013 10:...
16.0      [6/10/2013 9:00, 8/20/2013 18:00, 8/19/2013 18...
17.0                       [8/7/2013 17:45, 8/7/2013 17:45]
                                ...                        
1396.0    [10/25/2013 22:00, 10/28/2013 18:45, 10/25/201...
1397.0                     [11/3/2013 6:15, 11/3/2013 5:45]
1398.0    [11/20/2013 14:00, 9/3/2013 6:00, 11/21/2013 2...
1399.0    [8/5/2013 17:30, 7/23/2013 8:30, 7/23/2013 9:0...
1401.0    [10/18/2013 11:00, 10/19/2013 3:30, 10/12/2013...
Length: 560, dtype: object

In [14]:
def bookings_per_to_time(df):
    df = df.dropna()
    area_list = df.index
    d = {}
    for i in area_list:
        c = df.loc[i,'to_date']
        d[i] = c
    return d

bookings_per_time2 = bookings_per_to_time(area_based_df)
bookings_per_time2_series = pd.Series(bookings_per_time2)
bookings_per_time2_series

2.0       [nan, 7/20/2013 14:21, 8/18/2013 21:16, nan, n...
6.0       [6/29/2013 12:17, 1/2/2013 0:00, nan, nan, 7/3...
15.0             [7/17/2013 14:06, nan, nan, nan, nan, nan]
16.0      [nan, 8/20/2013 18:58, 8/19/2013 19:28, nan, nan]
17.0                        [8/7/2013 22:45, 8/8/2013 0:21]
                                ...                        
1396.0    [10/27/2013 23:59, 10/28/2013 23:59, 10/26/201...
1397.0                                           [nan, nan]
1398.0    [11/20/2013 15:28, 9/3/2013 23:59, 11/22/2013 ...
1399.0    [8/5/2013 18:02, 7/23/2013 9:34, 7/23/2013 10:...
1401.0    [10/18/2013 18:00, 10/19/2013 4:00, 10/12/2013...
Length: 560, dtype: object

**Finally, we can now merge each of those individual Series object into a DataFrame, rename some columns & then store it separately in two dataframes:**
1. 'final_df1' sorted based on 'no. of bookings'
2. 'final_df2' sorted based on 'No, of Cancellations' (both in descending order).

**We will then proceed with finding out common relations between these dataframes.**

In [15]:
dfs = pd.DataFrame([bookings_per_area_series, bookings_per_time1_series, bookings_per_time2_series, cancellation_per_area_series, no_of_cancelled_series, travel_ids_per_area_series])
dfs = dfs.T
dfs = dfs.rename(columns={0:'no. of bookings', 1:'from_date/time', 2:'to_date/time', 3:'Ever Cancelled?', 4:'No. of Cancels', 5:'Cancelled_Travel_types'})
final_df1 = dfs.sort_values('no. of bookings',ascending=False)
final_df1 = final_df1.reset_index()
final_df1.rename(columns={'index':'area_id'},inplace=True)
final_df1.head(20)

,area_id,no. of bookings,from_date/time,to_date/time,Ever Cancelled?,No. of Cancels,Cancelled_Travel_types
0,393.0,3858,"[5/18/2013 1:30, 9/29/2013 13:30, 11/22/2013 3...","[nan, nan, 11/22/2013 4:44, 9/25/2013 23:57, 8...",Yes,116,"[P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ..."
1,571.0,1631,"[4/7/2013 5:00, 11/8/2013 7:45, 5/21/2013 19:1...","[nan, 11/8/2013 11:30, nan, 8/8/2013 3:10, 11/...",Yes,127,"[P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ..."
2,293.0,1052,"[11/5/2013 17:00, 9/2/2013 19:30, 10/25/2013 1...","[11/5/2013 16:56, 9/2/2013 22:06, 10/25/2013 1...",Yes,99,"[P2P, H, H, P2P, P2P, P2P, P2P, P2P, H, P2P, P..."
3,585.0,911,"[11/23/2013 3:30, 8/3/2013 9:30, 7/11/2013 21:...","[11/23/2013 4:14, 8/3/2013 9:44, 7/11/2013 23:...",Yes,24,"[P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ..."
4,1010.0,768,"[10/11/2013 9:30, 1/17/2013 18:00, 4/5/2013 18...","[10/11/2013 10:36, nan, nan, 8/10/2013 23:30, ...",Yes,53,"[H, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2..."
5,142.0,727,"[10/27/2013 18:00, 10/29/2013 4:30, 11/21/2013...","[10/27/2013 19:50, 10/29/2013 6:07, 11/21/2013...",Yes,43,"[P2P, P2P, P2P, P2P, H, P2P, H, P2P, P2P, H, P..."
6,83.0,719,"[2/14/2013 9:00, 6/7/2013 11:00, 5/24/2013 22:...","[nan, nan, 5/25/2013 0:00, 8/20/2013 8:00, nan...",Yes,68,"[P2P, P2P, P2P, P2P, H, P2P, P2P, H, P2P, P2P,..."
7,1384.0,628,"[5/29/2013 8:00, 2/11/2013 9:30, 5/16/2013 9:4...","[nan, nan, nan, 8/20/2013 21:37, nan, 8/2/2013...",Yes,13,"[P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ..."
8,1096.0,542,"[7/28/2013 8:00, 10/15/2013 15:30, 8/5/2013 7:...","[7/28/2013 10:19, 10/16/2013 0:30, 8/5/2013 7:...",Yes,49,"[P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ..."
9,58.0,466,"[2/3/2013 11:00, 2/2/2013 8:15, 6/3/2013 5:00,...","[2/3/2013 0:00, nan, nan, 7/12/2013 22:05, 7/1...",Yes,37,"[P2P, H, P2P, P2P, P2P, P2P, P2P, P2P, H, P2P,..."


In [16]:
final_df2 = dfs.sort_values('No. of Cancels',ascending=False)
final_df2 = final_df2.reset_index()
final_df2.rename(columns={'index':'area_id'},inplace=True)
final_df2.head(20)

,area_id,no. of bookings,from_date/time,to_date/time,Ever Cancelled?,No. of Cancels,Cancelled_Travel_types
0,571.0,1631,"[4/7/2013 5:00, 11/8/2013 7:45, 5/21/2013 19:1...","[nan, 11/8/2013 11:30, nan, 8/8/2013 3:10, 11/...",Yes,127,"[P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ..."
1,393.0,3858,"[5/18/2013 1:30, 9/29/2013 13:30, 11/22/2013 3...","[nan, nan, 11/22/2013 4:44, 9/25/2013 23:57, 8...",Yes,116,"[P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ..."
2,293.0,1052,"[11/5/2013 17:00, 9/2/2013 19:30, 10/25/2013 1...","[11/5/2013 16:56, 9/2/2013 22:06, 10/25/2013 1...",Yes,99,"[P2P, H, H, P2P, P2P, P2P, P2P, P2P, H, P2P, P..."
3,83.0,719,"[2/14/2013 9:00, 6/7/2013 11:00, 5/24/2013 22:...","[nan, nan, 5/25/2013 0:00, 8/20/2013 8:00, nan...",Yes,68,"[P2P, P2P, P2P, P2P, H, P2P, P2P, H, P2P, P2P,..."
4,1010.0,768,"[10/11/2013 9:30, 1/17/2013 18:00, 4/5/2013 18...","[10/11/2013 10:36, nan, nan, 8/10/2013 23:30, ...",Yes,53,"[H, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2..."
5,1096.0,542,"[7/28/2013 8:00, 10/15/2013 15:30, 8/5/2013 7:...","[7/28/2013 10:19, 10/16/2013 0:30, 8/5/2013 7:...",Yes,49,"[P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ..."
6,142.0,727,"[10/27/2013 18:00, 10/29/2013 4:30, 11/21/2013...","[10/27/2013 19:50, 10/29/2013 6:07, 11/21/2013...",Yes,43,"[P2P, P2P, P2P, P2P, H, P2P, H, P2P, P2P, H, P..."
7,1330.0,362,"[5/19/2013 8:00, 5/19/2013 7:30, 3/2/2013 5:15...","[5/19/2013 0:00, nan, nan, nan, nan, nan, 7/4/...",Yes,41,"[P2P, P2P, L, P2P, P2P, P2P, P2P, P2P, P2P, P2..."
8,1371.0,327,"[5/29/2013 14:45, 11/13/2013 16:15, 11/19/2013...","[nan, 11/13/2013 17:09, 11/19/2013 6:48, 11/11...",Yes,39,"[P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, P2P, ..."
9,269.0,413,"[7/19/2013 16:45, 7/29/2013 16:45, 8/3/2013 10...","[7/19/2013 17:53, 7/29/2013 17:53, 8/3/2013 20...",Yes,39,"[P2P, P2P, P2P, P2P, P2P, H, P2P, P2P, P2P, P2..."


## Conclusion:
**From the two dataframes above we can see that most of the areas are COMMON in the top ranked rows in both these dataframes as they have the same area_id 's & these COMMON areas in top rows of both indicate that they have the highest number of bookings & a comparatively lower number of cancellations. So we can try to increase prices in those areas since the demand is peaking in those areas. Apart from that we can also deduce the following important results to classify the "non-problematic" & "problematic" areas from the dataframes above:**

**1. If we reduce the dataframes above to look at some top 20 rows of data in 'final_df1' & filter out those area_id's (& corresponding data) from final_df1 whose area_id's are not found in the corresponding top rows of data in final_df2 by using a "left or right join/merge" & we can consider them as mostly "non-problematic" areas since the cab bookings in those area is higher & they correspond to a much, much lower rate of cancellations (as they are not included in top rows of final_df2 which is sorted based on no. of cancellations). So for those area_ids we can safely increase the price to meet the demand. Also we can further try to divert some cabs working in these areas to the more "problematic" areas that are discussed in next point.**

**2. If we filter out those area_id's (& its data) from top rows of 'final_df2' that are not present in corresponding top rows of 'final_df1' we can consider that those areas are mostly 'problematic' since those areas have very higher rates of cancellations for comparatively lower no. of bookings. So, for those areas we can check out their 'from time' from the 'from_date/time' that corresponds to the most occuring cancelled_travel_type (which is mostly P2P as we can see from the dataframes above) & divert more cabs to those areas at that time in order to avoid more cancellations in those areas. Also we can try to REDUCE the prices in these most cancelled areas in order to get more bookings from these areas.**